In [31]:
import torch
from torch import nn, transpose, unsqueeze, squeeze, flatten
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchmetrics import AUROC
import pandas as pd
import numpy as np

In [11]:
# Loading datasets

training_labels = pd.read_csv('/home/sesso/Documents/Study/pytorch_learn/volcanoes_venus/venus_data/archive/volcanoes_train/train_labels.csv')
training_input = pd.read_csv('/home/sesso/Documents/Study/pytorch_learn/volcanoes_venus/venus_data/archive/volcanoes_train/train_images.csv',
                                header = None)

testing_labels = pd.read_csv('/home/sesso/Documents/Study/pytorch_learn/volcanoes_venus/venus_data/archive/volcanoes_test/test_labels.csv')
testing_input = pd.read_csv('/home/sesso/Documents/Study/pytorch_learn/volcanoes_venus/venus_data/archive/volcanoes_test/test_images.csv',
                                header = None)           

# Creating tensor dataset

target_col = 'Type'

# Make nan in targe col = 0 
training_labels[target_col] = training_labels[target_col].fillna(0)
testing_labels[target_col] = testing_labels[target_col].fillna(0)

# use 30% of training data for validation
train_set_size = int(len(training_input) * 0.7)
valid_set_size = len(training_input) - train_set_size
batch_size = 64

## Convert to numpy array

training_labels = training_labels[target_col].to_numpy()
training_input = training_input.to_numpy()

testing_labels = testing_labels[target_col].to_numpy()
testing_input = testing_input.to_numpy()

## Make tensors of them

training_inputs = torch.tensor(training_input, dtype=torch.float32)
training_targets = torch.tensor(training_labels, dtype=torch.long)

testing_inputs = torch.tensor(testing_input, dtype=torch.float32)
testing_targets = torch.tensor(testing_labels, dtype=torch.long)

## Tensor datasets

dtrain = TensorDataset(training_inputs, training_targets)
dtest = TensorDataset(testing_inputs, testing_targets)

## Split train and validation set

### split the train set into two
seed = torch.Generator().manual_seed(42)
train_set, valid_set = random_split(dtrain, [train_set_size, valid_set_size], generator=seed)

### Batch loader

train_dl = DataLoader(train_set, batch_size = batch_size, shuffle=True) 
val_dl = DataLoader(valid_set) 
test_dl = DataLoader(dtest, batch_size =batch_size)



In [3]:
for X, y in test_dl:
    print(f"Shape of X [N, C]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C]: torch.Size([64, 12100])
Shape of y: torch.Size([64]) torch.int64


https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

In [4]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
k_size = 101

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # self.flatten = nn.Flatten()
        self.seq = nn.Sequential(
            nn.Conv1d(1, 6, k_size), # -> 121000 - 101 + 1 = 120900 
            nn.ReLU(),
            nn.MaxPool1d(2), # -> 120900/2 = 60450
            nn.Conv1d(6, 16, k_size), # -> 60450 - 101 + 1 = 60350
            nn.ReLU(),
            nn.MaxPool1d(2), # -> 60350/2 = 30175
            nn.Flatten(1),
            nn.Linear(47200, 512),  
            # nn.fc2 = nn.Linear(120, 84)
            # nn.fc3 = nn.Linear(84, 10)
            # nn.Linear(110*110, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.Linear(10,5)
        )

    def forward(self, x):
        # x = self.flatten(x)
        # x = torch.transpose(x, 0, 1)
        x = torch.unsqueeze(x, dim=0)
        x = torch.transpose(x, 0, 1)
        # x.transpose(0,1)
        logits = self.seq(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (seq): Sequential(
    (0): Conv1d(1, 6, kernel_size=(101,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(6, 16, kernel_size=(101,), stride=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=47200, out_features=512, bias=True)
    (8): ReLU()
    (9): Linear(in_features=512, out_features=512, bias=True)
    (10): ReLU()
    (11): Linear(in_features=512, out_features=10, bias=True)
    (12): Linear(in_features=10, out_features=5, bias=True)
  )
)


In [5]:
# loss_fn = nn.CrossEntropyLoss()
from tkinter import Variable


loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader): # CHECK
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 40
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dl, model, loss_fn, optimizer)
    test(test_dl, model, loss_fn)
print("Done!")

In [93]:
torch.save(model.state_dict(), "/home/sesso/Documents/Study/pytorch_learn/volcanoes_venus/model/state_dict")

In [ ]:
# Loading the saved model

model.load_state_dict(torch.load("/home/sesso/Documents/Study/pytorch_learn/volcanoes_venus/model/state_dict"))
model.eval()

In [22]:
def eval_validation(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    cols = ['actual', 'pred', 'actual_volcano', 'pred_volcano']
    out_df = pd.DataFrame(columns=cols, index=range(size))
    a = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            out_df.loc[a].actual = y.item()
            out_df.loc[a].pred = pred.argmax(1).item()
            actual_volcano = y.item() > 0
            out_df.loc[a].actual_volcano = actual_volcano          
            pred_volcano = pred.argmax(1).item() > 0
            out_df.loc[a].pred_volcano = pred_volcano
            a += 1
    test_loss /= size
    correct /= size
    accuracy = 100*correct
    print(f"Test Error: \n Accuracy: {accuracy:>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return accuracy, test_loss, out_df

In [ ]:
# Test in validation set
accuracy, test_loss, out_df = eval_validation(val_dl, model, loss_fn)

In [24]:
out_df.head()

,actual,pred,actual_volcano,pred_volcano
0,0,0,False,False
1,0,0,False,False
2,0,0,False,False
3,0,0,False,False
4,3,0,True,False


In [29]:
out_df['TP'] = (out_df['actual_volcano'] == True) & (out_df['pred_volcano'] == True)
out_df['FP'] = (out_df['actual_volcano'] == False) & (out_df['pred_volcano'] == True)
out_df['TN'] = (out_df['actual_volcano'] == False) & (out_df['pred_volcano'] == False)
out_df['FN'] = (out_df['actual_volcano'] == True) & (out_df['pred_volcano'] == False)

In [44]:
preds = out_df['pred_volcano'].to_numpy().astype(int)
preds = torch.from_numpy(preds)
target = out_df['actual_volcano'].to_numpy().astype(int)
target = torch.from_numpy(target)
auroc = AUROC(num_classes=1)
auroc_metric = auroc(preds, target).item()


/home/sesso/anaconda3/envs/pytorch_learn/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [45]:
print(f"Accuracy: {accuracy:>0.1f}%\nLoss: {test_loss:>8f} \nAUROC: {auroc_metric:>0.2f}  ")

Accuracy: 87.6%
Loss: 0.479428 
AUROC: 0.82  


In [46]:
out_df.sum()/len(out_df)


actual                0.42
pred              0.395714
actual_volcano    0.145238
pred_volcano      0.130476
TP                0.099048
FP                0.031429
TN                0.823333
FN                 0.04619
dtype: object